In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import cartopy.crs as ccrs

In [3]:
def read_data(path):
    dsets = []
    for p in path:
        dsets.append(xr.open_dataset(p))
    return xr.merge(dsets)

In [4]:
hist_nor = read_data(snakemake.input.noresm_histSST).isel(time=slice(5,None))
piClim_nor = read_data(snakemake.input.noresm_piClim).isel(time=slice(5,None))
ec_hist = read_data(snakemake.input.ecEarth_histSST).isel(time=slice(5,None))
ec_piClim = read_data(snakemake.input.ecEarth_piClim).isel(time=slice(5,None))
nor_late = hist_nor.isel(time=slice(-30,None))
ec_late = ec_hist.isel(time=slice(-30,None))
ec_start = ec_hist.isel(time=slice(None,30))
nor_start = hist_nor.isel(time=slice(None,30))
# ec

In [5]:
hist_start = [ec_start, nor_start]
hist_late = [ec_late, nor_late]
piclim = [ec_piClim, piClim_nor]

In [16]:

def plot(da : xr.DataArray, ax, name,vmin=None, vmax=None, **mesh_kwargs):
    
    da.std(dim='time').plot(ax=ax, vmin=vmin, vmax=vmax,transform=ccrs.PlateCarree(), **mesh_kwargs)
    ax.set_title(name)
    ax.coastlines()



In [59]:
vname = snakemake.wildcards.get('vName', 'od550dust')
fig,ax = plt.subplots(figsize=(17,6),ncols=3, nrows=2, subplot_kw={'projection':ccrs.Robinson()})
cmap = cm.get_cmap('YlOrBr', 21).copy()
cmap.set_under('k', alpha=0.0)
for das, dal, pi,i in zip(hist_start, hist_late, piclim, range(2)):
    
    plot(das[vname], ax[i,0], name=das.source_id,add_colorbar=False, cmap=cmap, vmin=0.01, vmax=0.3)
    plot(dal[vname], ax[i,1], name=dal.source_id,add_colorbar=False, cmap=cmap, vmin=0.01, vmax=0.3)
    plot(pi[vname], ax[i,2], name=pi.source_id,add_colorbar=False, cmap=cmap, vmin=0.01, vmax=0.3)

fig.text(0.238, 0.95,'First 30 years \n histSST - histSST-piaer', ha='center', va='center')
fig.text(0.512, 0.95, 'Last 30 years \n histSST - histSST-piaer', ha='center', va='center')
fig.text(0.782, 0.95, '30 years \n piClim-aer - piClim-control', ha='center', va='center')
plt.suptitle(f'Variability in {vname}, histSST first 30 year, last 30 years, piClim', fontsize=16, y=1.06)
cax = fig.add_axes([0.33,0.02,0.36,0.06])
cb=fig.colorbar(cm.ScalarMappable(mpl.colors.Normalize(0.01,0.3),cmap=cmap),cax=cax, 
             orientation = 'horizontal', extend='min')
plt.savefig(snakemake.output.dod_comp, dpi=144, bbox_inches='tight')